# Homework 4

Introduction:
    Combining several methods is the improvement of baseline:<br>
        1) Using the HMM based on the IBM2 model after the convergence of IBM1 model<br>
        2) Using the smoothing probalities<br>
        3) Intersecting the alignments e -> f and f -> e<br>
        4) Filling missing alignment on the diagonal<br>
    The best AER is 0.143213. <br>


# Baseline


We first implemented baseline model which is the IBM1. It ends up AER approximately equal to 0.347 with n = 100000

# Intersection of e -> f and f -> e

We then improved the baseline by training two models. One is e->f, another is f->e. Each model generated a set of alignment, we choose the intersection of two sets. With this method, we have AER about 0.23 with n = 100000

# HMM based IBM2

We implemented IBM2 as suggested in the one of the paper.<br>
    We initilized the IBM2 with parameters from IBM1 after IBM1 is converged.<br>
    We initialized the t uniformly.<br>
    For each pair (i,j) where i and j are position index of source language and target language, we initialize the a to be the relative distance to the diagonal of the (i,j) plan. That is saying for a(i,j,J,I)= relative position of (i,j)/sqrt(I^2+J^2)<br>
    We then followed the regular step of IBM2. We collect the counts of parameters. Sum up each count respectively and normalize them.<br>
    Repeat this algorithm until it converges<br>
    <br>
We oberserved that without restrict the number of iteration to be at most 5, the IBM2 takes extremely long time(almost 1 and half an hour to finish). So we add the constrain that forces the loop to run at most 5 iteration(Although with this constrian, it has slightly worse AER score). By adding this, we got the AER around 0.19.
    

# Smoothing

We tried to implement add-n smoothing method for the IBM1 and IBM2 model. We found the AER is higher when we use the add-n smoothing method for IBM1 so that we use the add-n smoothihng method for IBM2 model only. We initialized n=0.01, V=100000, threshold=0.0001 and we get the AER 0.145413.As for improvement of the add-n smoothing method, we tried to different n and V to get lower AER. Currently, We found the best AER 0.143213 with n=0.005, V=100000 and threshold=0.00005.

# Intersection of p(f|e) and p(e|f)

An intuition will be, instead of giving a guess when an alignment is unsure, it only guess when the alignment is likely to be correct. 



# Diagonal Filling

The diagonal filling is very simple, if there is missing alignments between 2 consecutive computed alignments, and the 2 alignments are on the same diagonal(45 degree), then the algorithm assumes all missing alignments are on that diagonal(45 degree).


By the way, there is another method attempted but ended up worse than the above method. Which it only keeps alignments that are close to the global diagonal of the 2 sentences. The bad thing about it is this method does not work well with the intersection, since alignments that are not intersect will still be removed.


# Conclusion

So the final trainning algorithm is followed:<br>
1. Running IBM1 to get parameters of f->e until IBM1 converges
2. Using the parameters got from IBM1 to initiliz HMM based IBM2
3. Save the alignment of f->e
4. Apply the similar procedure(step 1 ~ 3) for f->e
6. Make a intersection of alignment(f->e) and alignment(e->f)
7. Fill the alignment with diagonal

In [1]:
# hw4_main.py
import sys, os, logging
from collections import defaultdict
from itertools import islice

import align as align


# setting params
datadir = 'data'
fileprefix = 'hansards'
english = 'en'
french = 'fr'
alignment = 'a'
threshold = 0.00005
num_sents = 100000
n = sys.maxsize

# reading data
f_data = "%s.%s" % (os.path.join(datadir, fileprefix), french)
e_data = "%s.%s" % (os.path.join(datadir, fileprefix), english)
a_data = "%s.%s" % (os.path.join(datadir, fileprefix), alignment)

sys.stderr.write("Training with Dice's coefficient...\n")

bitext = [[sentence.strip().split() for sentence in pair] for pair in islice(zip(open(f_data), open(e_data)), num_sents)]
bitextf = [[sentence.strip().split() for sentence in pair] for pair in islice(zip(open(e_data), open(f_data)), num_sents)]


A_fe = align.train(bitext, threshold, 'fe')
A_ef = align.train(bitextf, threshold, 'ef')


A = ()
for i in range(len(A_ef)):
    A += ( (set(A_fe[i]) & set(A_ef[i]) ), )


A = align.sortAlignment(A)
align.fillDiagonal(A)
align.writeAlignment(A, 'A_test')

inp = open('data/A_test.txt', 'r')

align.score_alignment(f_data, e_data, a_data, inp, n)



Training with Dice's coefficient...
[IBM 1 Iteration 1]  diff: 0.0010417233
[IBM 1 Iteration 2]  diff: 0.0007522341
[IBM 1 Iteration 3]  diff: 0.0003088043
[IBM 1 Iteration 4]  diff: 0.0000101301

[IBM 2 Iteration 5]  diff: 0.0123915338
[IBM 2 Iteration 6]  diff: 0.0046454828
[IBM 2 Iteration 7]  diff: 0.0097978306
[IBM 2 Iteration 8]  diff: 0.0048294450
[IBM 2 Iteration 9]  diff: 0.0022660884

[IBM 1 Iteration 1]  diff: 0.0076377058
[IBM 1 Iteration 2]  diff: 0.0022381828
[IBM 1 Iteration 3]  diff: 0.0049201463
[IBM 1 Iteration 4]  diff: 0.0004831940
[IBM 1 Iteration 5]  diff: 0.0000327091

[IBM 2 Iteration 6]  diff: 0.0106489386
[IBM 2 Iteration 7]  diff: 0.0081565380
[IBM 2 Iteration 8]  diff: 0.0106603646
[IBM 2 Iteration 9]  diff: 0.0034087665
[IBM 2 Iteration 10]  diff: 0.0011817730



0-0 1-1 

  Alignment 0  KEY: ( ) = guessed, * = sure, ? = possible
  ------
 |(*)    | 2
 |   (*) | .
  ------
   2  . 

0-0 1-1 

  Alignment 1  KEY: ( ) = guessed, * = sure, ? = possible
  ------
 |(*)    | 2
 |   (*) | .
  ------
   2  . 

0-0 1-1 

  Alignment 2  KEY: ( ) = guessed, * = sure, ? = possible
  ------------
 |(*)          | oh
 |   (*)       | ,
 |       *     | oh
 |          *  | !
  ------------
   o  ,  o  ! 
   h     h    

0-0 1-1 

  Alignment 3  KEY: ( ) = guessed, * = sure, ? = possible
  ------------
 |(*)          | oh
 |   (*)       | ,
 |       *     | oh
 |          *  | !
  ------------
   o  ,  o  ! 
   h     h    

0-0 1-1 

  Alignment 4  KEY: ( ) = guessed, * = sure, ? = possible
  ------------
 |(*)          | oh
 |   (*)       | ,
 |       *     | oh
 |          *  | !
  ------------
   o  ,  o  ! 
   h     h    

0-2 2-1 3-3 4-4 6-5 8-7 11-8 12-9 13-10 14-11 15-12 20-15 23-18 24-19 7-6 21-16 22-17 

  Alignment 5  KEY: ( ) = guessed, * = sure, ? 

 |                ?                                   | entendre
 |                  (*)                               | que
 |                     (*)       ?                    | un
 |                              (*)                   | programme
 |                            ?  ?                    | de
 |                           (*)                      | construction
 |                        (?)                         | plus
 |                         ?                          | considérable
 |                                 (?)                | peut
 |                                  ?                 | -
 |                                  ?                 | être
 |                                              ?     | une
 |                                              *     | poussée
 |                                          (*)       | inflationniste
 |                                                (*) | .
  ---------------------------------------------------
   M 

 |    ?     *                                | ai
 |      (*)                                  | pas
 |             ?                             | de
 |               (*)                         | scrupule
 |                  (?)                      | quant
 |                   ?                       | à
 |                     (*)                   | le
 |                        (*)                | genre
 |                           (*)             | de
 |                              (*)          | langage
 |                                           | que
 |                                  *        | je
 |                                     ?     | tiens
 |                                       (*) | .
  ------------------------------------------
   i  d  n  h  a  q  a  t  t  o  l  I  u  . 
      o  o  a  n  u  b  h  y  f  a     s    
         t  v  y  a  o  e  p     n     e    
            e     l  u     e     g          
                  m  t           u          
          

 |                                     *        | de
 |                                        ?     | le
 |                                        *     | Office
 |                                        *     | canadien
 |                                        *     | de
 |                                        *     | commercialisation
 |                                    ( ) *     | de
 |                                        *     | les
 |                                       (*)    | oeufs
 |                                          (*) | .
  ---------------------------------------------
   i  D  t  w  a  a  t  B  C  w  b  w  f  C  . 
   n  e  h  a  n  n  h  r  o  o  e  i  r  E    
      c  e  s     n  a  i  l  u     t  o  M    
      e  r        o  t  t  u  l     h  m  A    
      m  e        u     i  m  d     d          
      b           n     s  b        r          
      e           c     h  i        a          
      r           e        a        w          
          

...

 |                        (*) ?  ?  ?     | Atlantique
 |                                    (*) | .
  ---------------------------------------
   o  c  h  t  o  d  f  t  A  p  e  c  . 
   u  a  e  h  t  a  r  h  t  r  c  o    
   r  u  a  e  h  y  o  e  l  o  o  u    
      c  r     e     m     a  v  n  n    
      u  d     r           n  i  o  c    
      s                    t  n  m  i    
                           i  c  i  l    
                           c  e  c       
                              s          

0-0 7-6 8-7 

  Alignment 377  KEY: ( ) = guessed, * = sure, ? = possible
  ------------------------
 |(?) ?  ?  ?  ?  ?  ?     | faut
 | ?  ?  ?  ?  ?  ?  ?     | pas
 | ?  ?  ?  ?  ?  ?  ?     | nous
 | ?  ?  ?  ?  ?  ?  ?     | prendre
 | ?  ?  ?  ?  ?  ?  ?     | pour
 | ?  ?  ?  ?  ?  ?  ?     | de
 | ?  ?  ?  ?  ?  ?  ?     | les
 | ?  ?  ?  ?  ?  ? (?)    | poires
 |                     (*) | !
  ------------------------
   c  o  ,  g  u  a  b  ! 
   o  n     i  s   

 |                            ?  ?  ?  ?  ?                                                  | que
 |      ( )                   ?  ?  ?  ?  ?                                                  | il
 |                            ?  ?  ?  ?  ?                                                  | faudrait
 |                            ? (?) ?  ?  ?                                                  | avoir
 |                            ?  ?  ?  ?  ?                                                  | servi
 |                                           ?  ?  ?  ?  ?  ?  ?  ?                          | plus
 |                                           ?  ?  ?  ?  ?  *  ?  ?                          | de
 |                                                    ?  ?  ? (*) ?                          | 365
 |                                                    ?  ?  ?  ? (*)                         | jours
 |                                                                   *                       | à
 |

 |                                             (*) | .
  ------------------------------------------------
   w  a  a  s  c  a  t  d  a  i  o  t  s  o  b  . 
   e  s     o  a  f  o  i  m  n  n  h  o  f  a    
            c  n  f     s  o  d     a  r     s    
            i  n  o     c  n  i     t  t     i    
            e  o  r     r  g  v              s    
            t  t  d     i     i                   
            y           m     d                   
                        i     u                   
                        n     a                   
                        a     l                   
                        t     s                   
                        e                         

0-0 1-6 5-3 6-15 8-5 9-9 11-11 12-12 14-14 17-16 18-17 19-18 22-21 23-23 10-10 13-13 20-19 21-20 

  Alignment 412  KEY: ( ) = guessed, * = sure, ? = possible
  ------------------------------------------------------------------------
 |( )       ?  ?  ?  ?  ?  ?                   

 |                        (*)                                     | et
 |                            ?  ?  ?  ?  ?  ?  ?                 | celle
 |                            ?  ?  ?  ?  ?  ?  ?                 | de
 |                            ?  ?  ?  ?  ?  *  ?                 | ses
 |                            ?  ?  ?  ?  ?  ?  ?                 | hauts
 |                            ?  ?  ?  ?  ?  ? (*)                | fonctionnaires
 |                                                 ?  ?  ?        | devrait
 |                                                 ?  ?  ?        | faire
 |                                                 ?  ?  ?        | le
 |                                                (?) ?  ?        | objet
 |                                                 ? (?) ?        | de
 |                                                 ?  ? (?) ?     | une
 |                                                 ?  ?  ? (*)    | enquête
 |                                       

 |      (*)                                        | ,
 |          ?  ?  ?  *  ?                 ?  ?     | la
 |          ?  ?  ?  ? (*)                ?  ?     | question
 |                         ?  *                    | de
 |                               *                 | le
 |                                  *  *           | député
 |          ?  ?  ?  ?  ?                 ?  ?     | part
 |          ?  ?  ?  ?  ?                 ?  ?     | de
 |          ?  ?  ?  ?  ?                 ?  ?     | une
 |          ? (?) ?  ?  ?                 ?  ?     | hypothèse
 |          ?  ?  ?  ?  ?                 ?  ?     | toute
 |          ?  ?  ?  ?  ?                 ? (?)    | gratuite
 |                                             (*) | .
  ------------------------------------------------
   M  S  ,  t  p  o  t  q  p  b  t  H  M  i  w  . 
   r  p     h  r  f  h  u  u  y  h  o  e  s  r    
   .  e     e  e     e  e  t     e  n  m     o    
      a        m        s           .  b 